In [15]:
import pandas as pd
import numpy as np
from pingouin import cronbach_alpha
from scipy.stats import ttest_ind, f_oneway   

In [16]:
df = pd.read_csv("./resultados_avaliacao.csv")

In [ ]:
def teste_t_student(serie1, serie2, grupo1, grupo2):
    if len(serie1) > 1 and len(serie2) > 1:
        t_stat, p_val_t = ttest_ind(serie1.dropna(), serie2.dropna(), equal_var=False)
        sig = "significativa" if p_val_t < 0.05 else "não significativa"
        print(f"Teste t-Student entre {grupo1} e {grupo2}: t={t_stat:.2f}, p-valor={p_val_t:.4f} ({sig})")
    else:
        print(f"Dados insuficientes para o teste t-Student entre {grupo1} e {grupo2}")

In [32]:
def extrai_resposta_por_grupo(df, data_grupo1, data_grupo2, data_grupo3): 
    df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y %H:%M:%S')
    
    questoes = [
        'Visibilidade do estado do sistema', 
        'Correspondência entre o sistema e o mundo real', 
        'Controle e liberdade do usuário', 
        'Consistência e padrões', 
        'Prevenção de erros', 
        'Reconhecimento em vez de memorização', 
        'Flexibilidade e eficiência de uso', 
        'Design estético e minimalista', 
        'Ajude os usuários a reconhecer, diagnosticar e corrigir erros', 
        'Ajuda e documentação'
    ]
    
    mapeamento_respostas = {
        'Concordo Totalmente': 5,
        'Concordo': 4,
        'Indiferente': 3,
        'Discordo': 2,
        'Discordo Totalmente': 1
    }
    
    def extrai_respostas(df):
        heuristicas_respostas = {heuristica: [] for heuristica in questoes}
        for heuristica in questoes:
            for coluna in df.columns:
                if heuristica in coluna:
                    contagem_respostas = df[coluna].value_counts()
                    respostas = []
                    for categoria, valor in mapeamento_respostas.items():
                        if categoria in contagem_respostas:
                            respostas += [valor] * contagem_respostas[categoria]
                    heuristicas_respostas[heuristica].append(respostas)
        return heuristicas_respostas

    grupo1 = df[df['data'].dt.strftime('%d/%m/%Y').isin(data_grupo1)]
    grupo2 = df[df['data'].dt.strftime('%d/%m/%Y').isin(data_grupo2)]
    grupo3 = df[df['data'].dt.strftime('%d/%m/%Y').isin(data_grupo3)]

    media_grupo1 = grupo1['4. Em qual ano do curso está?'].mean()
    media_grupo2 = grupo2['4. Em qual ano do curso está?'].mean()
    media_grupo3 = grupo3['4. Em qual ano do curso está?'].mean()

    media_idade_grupo1 = grupo1['2. Qual a sua idade?'].mean()
    media_idade_grupo2 = grupo2['2. Qual a sua idade?'].mean()
    media_idade_grupo3 = grupo3['2. Qual a sua idade?'].mean()

    disciplinas = ["Construção de Software", "Gerenciamento de Projetos de Software", "Interação Humano-Computador"]


    print(f"Media de ano do curso grupo {disciplinas[0]} {media_grupo1}")
    print(f"Media de ano do curso grupo {disciplinas[1]} {media_grupo2}")
    print(f"Media de ano do curso grupo {disciplinas[2]} {media_grupo3}")

    print(f"Media de idade do grupo {disciplinas[0]} {media_idade_grupo1}")
    print(f"Media de idade do grupo {disciplinas[1]} {media_idade_grupo2}")
    print(f"Media de idade do grupo {disciplinas[2]} {media_idade_grupo3}")

    
    respostas_grupo1 = extrai_respostas(grupo1)
    respostas_grupo2 = extrai_respostas(grupo2)
    respostas_grupo3 = extrai_respostas(grupo3)
    
    # Análise de correlação e testes estatísticos entre os grupos para cada heurística
    for heuristica in questoes:
        respostas_g1 = respostas_grupo1[heuristica][0] if len(respostas_grupo1[heuristica]) > 0 else []
        respostas_g2 = respostas_grupo2[heuristica][0] if len(respostas_grupo2[heuristica]) > 0 else []
        respostas_g3 = respostas_grupo3[heuristica][0] if len(respostas_grupo3[heuristica]) > 0 else []
        
        # Convertendo para Series do pandas para análise
        serie_g1 = pd.Series(respostas_g1)
        serie_g2 = pd.Series(respostas_g2)
        serie_g3 = pd.Series(respostas_g3)
        
        print(f"\nAnálise para a heurística '{heuristica}':")

        teste_t_student(serie_g1, serie_g2, disciplinas[0], disciplinas[1])
        teste_t_student(serie_g2, serie_g3, disciplinas[1], disciplinas[2])
        teste_t_student(serie_g1, serie_g3, disciplinas[0], disciplinas[2])

        # Teste ANOVA entre os três grupos se houver dados suficientes
        if len(serie_g1) > 1 and len(serie_g2) > 1 and len(serie_g3) > 1:
            f_stat, p_val_anova = f_oneway(serie_g1.dropna(), serie_g2.dropna(), serie_g3.dropna())
            sig_anova = "significativa" if p_val_anova < 0.05 else "não significativa"
            print(f"ANOVA entre os três grupos: F={f_stat:.2f}, p-valor={p_val_anova:.4f} ({sig_anova})")


In [33]:
data_grupo1 = ['23/10/2024'] # Construção 
data_grupo2 = ['24/10/2024'] # Gerenciamento 
data_grupo3 = ['22/10/2024', '25/10/2024'] # IHC

extrai_resposta_por_grupo(df, data_grupo1, data_grupo2, data_grupo3)

Media de ano do curso grupo Construção de Software 3.0
Media de ano do curso grupo Gerenciamento de Projetos de Software 4.208333333333333
Media de ano do curso grupo Interação Humano-Computador 2.1875
Media de idade do grupo Construção de Software 20.8125
Media de idade do grupo Gerenciamento de Projetos de Software 22.125
Media de idade do grupo Interação Humano-Computador 20.03125

Análise para a heurística 'Visibilidade do estado do sistema':
Teste t-Student entre Construção de Software e Gerenciamento de Projetos de Software: t=-2.23, p-valor=0.0360 (significativa)
Teste t-Student entre Gerenciamento de Projetos de Software e Interação Humano-Computador: t=0.24, p-valor=0.8114 (não significativa)
Teste t-Student entre Construção de Software e Interação Humano-Computador: t=-2.06, p-valor=0.0501 (não significativa)
ANOVA entre os três grupos: F=3.73, p-valor=0.0289 (significativa)

Análise para a heurística 'Correspondência entre o sistema e o mundo real':
Teste t-Student entre Con